In [26]:
!pip install flask-ngrok

import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
import numpy as np
import math
import torch.optim as opt
import torch.nn.functional as F

#CUDA_VISIBLE_DEVICES=""

class ClassificationTrainDataset(Dataset):

    def __init__(self):
        # Inicijalizacija ili preuzimanje podataka  
        # Čitanje .csv fajla uz pomoć numpzy
        xy = np.loadtxt('classification_train.csv', delimiter=',', dtype=np.float32, skiprows=1)
        self.br_uzoraka = xy.shape[0]
      
        #prva kolona je ciljana vrednost, ostalo su svojstva
        self.x_data = torch.from_numpy(xy[:, 1:]) # [br_uzoraka, br_svojstava]
        self.y_data = torch.from_numpy(xy[:, [0]]) # [br_uzoraka, 1]

    # dataset[i] vraća i-ti uzorak
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # pozivom len(dataset) vraća veličinu skupa podataka
    def __len__(self):
        return self.br_uzoraka

# kreirati objekat zapredstavljanje skupa podataka obuke
train_set = ClassificationTrainDataset()


class ClassificationTestDataset(Dataset):

    def __init__(self):
        # Inicijalizacija ili preuzimanje podataka  
        # Čitanje .csv fajla uz pomoć numpzy
        xy = np.loadtxt('classification_test.csv', delimiter=',', dtype=np.float32, skiprows=1)
        self.br_uzoraka = xy.shape[0]
      
        #prva kolona je ciljana vrednost, ostalo su svojstva
        self.x_data = torch.from_numpy(xy[:, 1:]) # [br_uzoraka, br_svojstava]
        self.y_data = torch.from_numpy(xy[:, [0]]) # [br_uzoraka, 1]

    # dataset[i] vraća i-ti uzorak
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # pozivom len(dataset) vraća veličinu skupa podataka
    def __len__(self):
        return self.br_uzoraka


# Čitamo ceo skup podataka uz pomoč DataLoader
# shuffle: promešati podatke - dobro za obuku 
# num_workers: brže se učitava za više procesora
train_loader = DataLoader(dataset=train_set,
                          batch_size=3,
                          shuffle=True,
                          num_workers=2)


class MyNeuralNetwork(torch.nn.Module):
    def __init__(self, input_size):
        super(MyNeuralNetwork, self).__init__()

        self.input_size = input_size

        self.linear_1 = torch.nn.Linear(input_size, 20)
        self.linear_2 = torch.nn.Linear(20, 10)
        self.linear_3 = torch.nn.Linear(10, 1)

    def forward(self, x):
        out = F.relu(self.linear_1(x))
        out = F.relu(self.linear_2(out))
        out = F.sigmoid(self.linear_3(out))
        return out

mreza=MyNeuralNetwork(4)

optimizer=opt.SGD(mreza.parameters(), lr=0.003)
f_greske=torch.nn.BCELoss()

# Petlja za obuku
br_epoha = 15
ukupno_uzoraka = len(train_set)

for epoch in range(br_epoha):
    for i, (ulazi, izlazi) in enumerate(train_loader):
        predikcije=mreza(ulazi)
        gubici=f_greske(predikcije,izlazi)
       # print('Ocekivano', izlazi)
       # print('Predikcija', predikcije)
        gubici.backward()
        optimizer.step()
        mreza.zero_grad()

test_set = ClassificationTestDataset()
test_loader = DataLoader(dataset=test_set,
                          batch_size=1,
                          shuffle=True,
                          num_workers=1)
ukupno_test=len(test_set)
print('Ukupno test', ukupno_test)

for i, (ulazi, izlazi) in enumerate(test_loader):
  predikcije=mreza(ulazi)
  print('Ulazi',ulazi)
  print('Ocekivano', izlazi)
  print('Predikcija', predikcije)
  print('Greska', abs(predikcije-izlazi))



from flask import Flask
from flask_ngrok import run_with_ngrok
from flask import request
  
app = Flask(__name__)
run_with_ngrok(app)
  

@app.route("/predict")
def classification():
    potrosnja = float(request.args.get('potrosnja'))
    osobe = float(request.args.get('osobe'))
    sezona = float(request.args.get('sezona'))
    soba = float(request.args.get('soba'))


    ulazi=torch.tensor([[potrosnja, osobe, sezona, soba]])
    predikcije=mreza(ulazi)
    odgovor=str(predikcije[0].item())
    return odgovor
  
if __name__ == "__main__":
  app.run()

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1805: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Ukupno test 11
Ulazi tensor([[600.,   1.,   1.,   3.]])
Ocekivano tensor([[1.]])
Predikcija tensor([[0.6783]], grad_fn=<SigmoidBackward>)
Greska tensor([[0.3217]], grad_fn=<AbsBackward>)
Ulazi tensor([[900.,   2.,   1.,   3.]])
Ocekivano tensor([[1.]])
Predikcija tensor([[0.7760]], grad_fn=<SigmoidBackward>)
Greska tensor([[0.2240]], grad_fn=<AbsBackward>)
Ulazi tensor([[1000.,    1.,    0.,    3.]])
Ocekivano tensor([[1.]])
Predikcija tensor([[0.8063]], grad_fn=<SigmoidBackward>)
Greska tensor([[0.1937]], grad_fn=<AbsBackward>)
Ulazi tensor([[300.,   3.,   1.,   1.]])
Ocekivano tensor([[0.]])
Predikcija tensor([[0.5553]], grad_fn=<SigmoidBackward>)
Greska tensor([[0.5553]], grad_fn=<AbsBackward>)
Ulazi tensor([[50.,  1.,  0.,  1.]])
Ocekivano tensor([[0.]])
Predikcija tensor([[0.4564]], grad_fn=<SigmoidBackward>)
Greska tensor([[0.4564]], grad_fn=<AbsBackward>)
Ulazi tensor([[200.,   4.,   0.,   2.]])
Ocekivano tensor([[0.]])
Predikcija tensor([[0.5106]], grad_fn=<SigmoidBackward>)
Gr

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://d775-35-203-161-204.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
